# Old embedder:

In [ ]:
from openai import OpenAI
import tiktoken
import os
import json
import numpy as np

gen_model = 'gpt-4-turbo-preview'
embed_model = 'text-embedding-3-small'
text_encoder = 'cl100k_base'
max_gen_tokens = 4096
max_input_tokens = 128000
default_temperature = 0
max_embed_tokens = 8191

client = OpenAI()
encoder = tiktoken.get_encoding(text_encoder)

class Embedder1:
    def __init__(self, cache, model=embed_model, encoder=text_encoder, max_tokens=max_embed_tokens) -> None:
        self.model = model
        self.encoder = tiktoken.get_encoding(encoder)
        self.max_tokens = max_tokens
        self.cache = cache
        if not os.path.exists(cache):
            os.makedirs(cache)

    def encode_all(self, texts):
        final_embeddings = [None] * len(texts)
        new_texts  = []
        for ix, text in enumerate(texts):
            text = text.replace("\n", " ")
            hsh = hash(text)
            path = os.path.join(self.cache, f'{hsh}.txt')
            if not os.path.exists(path):
                new_texts.append((ix, text))
            else:
                final_embeddings[ix] = np.array([float(x) for x in open(path, 'r').read().split('\n') if len(x) > 0])
        print(f'Got {len(new_texts)} new texts')
        # split into batches of 2000
        for i in range(0, len(new_texts), 2000):
            batch = new_texts[i:i+2000]
            batch_texts = [x[1] for x in batch]
            embeddings = [x.embedding for x in client.embeddings.create(input = batch_texts, model=self.model).data]
            for j, (ix, text) in enumerate(batch):
                hsh = hash(text)
                path = os.path.join(self.cache, f'{hsh}.txt')
                np.savetxt(path, embeddings[j], delimiter=',')
                final_embeddings[ix] = np.array(embeddings[j])
        return np.array(final_embeddings)

    def encode(self, text):
        text = text.replace("\n", " ")
        hsh = hash(text)
        path = os.path.join(self.cache, f'{hsh}.txt')
        if os.path.exists(path):
            return np.array([float(x) for x in open(path, 'r').read().split('\n') if len(x) > 0])
        else:
            tokens = len(self.encoder.encode(text))
            if tokens > self.max_tokens:
                text = text[:self.max_tokens]
                print('Truncated text to max tokens')
            try:
                embedding = client.embeddings.create(input = [text], model=self.model).data[0].embedding
                np.savetxt(path, embedding, delimiter=',')
                return np.array(embedding)
            except:
                print(f'Error embedding text: {text}')
                return None

def create_embedder(cache, model=embed_model, encoder=text_encoder, max_tokens=max_embed_tokens):
    return Embedder(cache, model, encoder, max_tokens)

In [3]:
import duckdb
con = duckdb.connect(":default:")
con.sql("SELECT * FROM tbl")

CatalogException: Catalog Error: Table with name tbl does not exist!
Did you mean "information_schema.tables"?

In [8]:
import numpy as np
exists = [('[ 0.04866408  0.00642491 -0.00871138 ...  0.00014402  0.01314187\n -0.00945217]',)]
print(exists[0][0])
oib = exists[0][0]
print('oib', oib)
oab = np.fromstring(oib, sep=' ')
print(type(oab))

[
oib [


C:\Users\ddesouza\AppData\Local\Temp\ipykernel_24456\863069983.py:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  oab = np.fromstring(oib, sep=' ')
